# Embeddings

## Word2Vec

Мы рассмотрели векторные модели, так называемые count-based или *счётные*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые основные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)

Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.

### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $u$ и вектор контекста с помощью матрицы $v$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстах. Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл (агрегированные вектора в plain-text)
* .bin.gz — бинарник (вектора и параметры модели)

Загружаются они с помощью одного и того же гласса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [ ]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
urllib.request.urlretrieve(
    "http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", 
    "ruscorpora_mystem_cbow_300_2_2015.bin.gz"
)

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x7f1bcc9632e8>)

In [ ]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
# можно посмотреть словарь модели и как выглядят слова в нем  
for n, word in enumerate(model_ru.vocab):
  if n < 10:
      print(word)

это_SPRO
весь_APRO
человек_S
мочь_V
год_S
сказать_V
время_S
говорить_V
становиться_V
знать_V


In [ ]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [ ]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S 0.7165195941925049
месяц_S 0.631048858165741
вечер_S 0.5828739404678345
утро_S 0.5676207542419434
час_S 0.5605547428131104
минута_S 0.5297019481658936
гекатомбеон_S 0.4897990822792053
денек_S 0.48224714398384094
полчаса_S 0.48217129707336426
ночь_S 0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S 0.6946247816085815
утро_S 0.57301926612854
ноченька_S 0.5582467317581177
рассвет_S 0.5553582906723022
ночка_S 0.5351512432098389
полдень_S 0.5334426164627075
полночь_S 0.478694349527359
день_S 0.4780748784542084
сумерки_S 0.4390218257904053
фундерфун_S 0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681  0.04785006]
женщина_S 0.5979775190353394
парень_S 0.4991787

Находим косинусную близость пары слов:

In [ ]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.23895611


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [ ]:
print(model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


In [ ]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'ананас_S'

### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [ ]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

--2021-02-08 16:01:30--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67281491 (64M) [text/plain]
Saving to: ‘unlabeledTrainData.tsv’

unlabeledTrainData. 100%[===================>]  64.16M   288MB/s    in 0.2s    

2021-02-08 16:01:31 (288 MB/s) - ‘unlabeledTrainData.tsv’ saved [67281491/67281491]



In [ ]:
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [ ]:
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [ ]:
sentences = []  

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [ ]:
print(len(sentences))
print(sentences[0])

528987
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [ ]:
# это понадобится нам позже

with open('clean_text.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [ ]:
print("Training model...")

%time model_en = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 3min 22s, sys: 635 ms, total: 3min 22s
Wall time: 1min 46s


Смотрим, сколько в модели слов.

In [ ]:
print(len(model_en.wv.vocab))

28308


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [ ]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7638259530067444)]
[('men', 0.6104907989501953)]
[('europe', 0.7682209014892578), ('greece', 0.738439679145813), ('region', 0.7286756634712219)]
novel


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейс модели, чтобы потом сравнить результат с дообученной.

In [ ]:
model_en.wv.similarity('lion', 'rabbit')

0.29338896

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [ ]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2021-02-08 16:21:18--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167631 (164K) [text/plain]
Saving to: ‘alice.txt’

alice.txt           100%[===================>] 163.70K  --.-KB/s    in 0.003s  

2021-02-08 16:21:18 (60.3 MB/s) - ‘alice.txt’ saved [167631/167631]



In [ ]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [ ]:
model_path = "movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

Saving model...


In [ ]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(97239, 150225)

Лев и кролик стали ближе друг к другу!

In [ ]:
model.wv.similarity('lion', 'rabbit')

0.30253717

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [ ]:
model.init_sims(replace=True)
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

Saving model...


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Например, мы сделали свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные бенчмарки для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии, а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификатор мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Посчитаем качество нашей модели русской НКРЯ на датасете аналогий:

In [ ]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

--2021-02-08 16:27:36--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871776 (851K) [text/plain]
Saving to: ‘ru_analogy_tagged.txt’

ru_analogy_tagged.t 100%[===================>] 851.34K  --.-KB/s    in 0.02s   

2021-02-08 16:27:37 (49.0 MB/s) - ‘ru_analogy_tagged.txt’ saved [871776/871776]



In [ ]:
res = model_ru.accuracy('ru_analogy_tagged.txt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.evaluate_word_analogies() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
res[4]["section"]

'family'

In [ ]:
print(res[4]['incorrect'][:10])

[('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ДЕД_S', 'БАБКА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'КОРОЛЬ_S', 'КОРОЛЕВА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПРИНЦ_S', 'ПРИНЦЕССА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ДЕД_S', 'БАБКА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('ПАПА_S', 'МАМА_S', 'ДЕД_S', 'БАБКА_S'), ('ПАПА_S', 'МАМА_S', 'ОТЧИМ_S', 'МАЧЕХА_S')]


## Визуализация

На полученную модель можно посмотреть, визуализировав ее, например, на плоскости.
### t-SNE

**Стохастическое вложение соседей с t-распределением (t-SNE)**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). 

t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).
Алгоритм состоит из двух шагов:
1) сначала t-SNE создаёт распределение вероятностей по парам объектов высокой размерности таким образом, что похожие объекты будут выбраны с большой вероятностью, в то время как вероятность выбора непохожих точек будет мала
2) затем t-SNE определяет похожее распределение вероятностей по точкам в пространстве малой размерности и минимизирует расстояние Кульбака — Лейблера между двумя распределениями с учётом положения точек.

Изобразим на плоскости 1000 самых частотных слов из коллекции текстов про кино:

In [ ]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []

# создаем из предложений частотный словарь 
fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])

print(top_words[:50:])
top_words_vec = model[top_words]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'with', 'for', 'movie', 'but', 'film', 'you', 't', 'on', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'they', 'at', 'by', 'who', 'an', 'from', 'so', 'like', 'there', 'or', 'her', 'just', 'about', 'out', 'has', 'if', 'what', 'some', 'good', 'can']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [ ]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 12.6 s, sys: 53.2 ms, total: 12.7 s
Wall time: 6.76 s


In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Чтобы вычислить преобразование t-SNE быстрее и эффективнее, можно (и даже нужно) снизить размерность исходных данных с помощью алгоритма снижения размерности. Для разреженных данных использовать, SVD, для не разреженных PCA; а затем уже применять t-SNE:

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_50 = TruncatedSVD(n_components=50)
top_words_vec_50 = svd_50.fit_transform(top_words_vec)
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

In [ ]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)



In [ ]:
# можно так же через gensim работать 
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.simple.zip
# !unzip wiki.simple.zip

from gensim.models.wrappers import FastText

ftmodel = FastText.load_fasttext_format('wiki.simple')

print(ftmodel.most_similar('vectr'))

# Для моделей fasttext с русвекторес
# ftmodel = gensim.models.KeyedVectors.load("model.model")
# не используются частеречные теги

[('vectra', 0.8145402669906616), ('vec', 0.7235217094421387), ('vector', 0.6860759854316711), ('vectors', 0.6219983100891113), ('\\vec', 0.5633634924888611), ('vw', 0.5600247383117676), ('veckan', 0.5583956241607666), ('mpv', 0.5490771532058716), ('eigenvector', 0.5474075078964233), ('nevers_riv_nievre_', 0.5386427640914917)]


Есть отдельная одноименная библиотека `fasttext` для питона. 

In [ ]:
! git clone https://github.com/facebookresearch/fastText.git
! pip install fastText/.

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 39.92 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3084957 sha256=f3ab805e0b36df9eb12188b5426382696bd6ce9f7e9223c9cebe22f71bb4f1c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ro87qbmy/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [ ]:
import fasttext

# так можно обучить свою модель 
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

In [ ]:
ft_model.get_word_vector("movie")

array([-3.57243493e-02,  1.58063918e-01, -1.31975710e-01, -4.58512418e-02,
        1.07560731e-01,  2.17444181e-01, -1.32337660e-01,  2.36562882e-02,
        2.35748856e-04,  2.43025094e-01,  3.16724479e-02,  1.34295747e-01,
       -3.92858200e-02, -1.53035549e-02,  3.50480936e-02,  5.05375341e-02,
        3.09894625e-02, -2.97354937e-01, -1.11680967e-03,  1.60186350e-01,
        9.39910933e-02,  6.12168089e-02,  7.73812756e-02, -2.73895264e-02,
       -2.06161775e-02, -6.11183990e-04,  1.21513978e-01, -1.09616116e-01,
       -1.08716406e-01, -4.59789261e-02,  7.11279437e-02,  1.83442477e-02,
        1.14463326e-02,  6.98599890e-02, -5.05972980e-03,  6.63157851e-02,
       -6.26043826e-02, -1.02846317e-01, -1.00384094e-01, -4.58281077e-02,
        6.26604706e-02,  1.24450661e-01, -7.05418810e-02, -2.00940818e-02,
        9.89525095e-02, -5.13858572e-02,  1.43190414e-01, -8.29519033e-02,
        5.86603582e-02,  5.74491732e-02,  5.35967126e-02, -8.11504871e-02,
       -4.13022004e-02, -

In [ ]:
ft_model.get_nearest_neighbors('actor')

[(0.9999606013298035, 'actors'),
 (0.9999364018440247, 'attractive'),
 (0.9999338984489441, 'fact'),
 (0.9999316334724426, 'actual'),
 (0.9999226331710815, 'display'),
 (0.9999191761016846, 'terrific'),
 (0.9999188780784607, 'battle'),
 (0.9999170899391174, 'israel'),
 (0.9999163746833801, 'british'),
 (0.9999160170555115, 'predator')]

In [ ]:
ft_model.get_analogies("woman", "man", "actor")

[(0.999938428401947, 'act'),
 (0.9998956918716431, 'exactly'),
 (0.9998955726623535, 'actress'),
 (0.999885082244873, 'seemingly'),
 (0.9998830556869507, 'terrible'),
 (0.9998824596405029, 'surprisingly'),
 (0.9998821020126343, 'believable'),
 (0.9998811483383179, 'double'),
 (0.9998807907104492, 'written'),
 (0.9998795986175537, 'cable')]

In [ ]:
# проблема с опечатками решена

ft_model.get_nearest_neighbors('actr')

[(0.9999391436576843, 'act'),
 (0.9998903274536133, 'actors'),
 (0.9998863339424133, 'actor'),
 (0.9998792409896851, 'actress'),
 (0.9998623728752136, 'single'),
 (0.9998517632484436, 'actual'),
 (0.9998226761817932, 'terrible'),
 (0.9998196363449097, 'exact'),
 (0.9998190402984619, 'plot'),
 (0.9998172521591187, 'wrong')]

In [ ]:
# проблема с out of vocabulary словами - тоже

ft_model.get_nearest_neighbors('moviegeek')

[(0.9999324679374695, 'reviews'),
 (0.9999246597290039, 'review'),
 (0.9999151825904846, 'recommended'),
 (0.9999132752418518, 'rented'),
 (0.9998916387557983, 'waste'),
 (0.999889075756073, 'movie'),
 (0.9998835921287537, 'thank'),
 (0.9998812079429626, 'not'),
 (0.9998751878738403, 'watchable'),
 (0.9998645782470703, 'only')]

Кроме этого, fastText можно использовать для классификации, для этого нужен следующий формат размеченных данных:

__label_1__  text_1

__label_2__  text_2

...

Можно посмотреть примеры [тут](https://fasttext.cc/docs/en/supervised-tutorial.html) 

Есть встроенные функции:

    load_model(path)
        Load a model given a filepath and return a model object.

    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens

    train_supervised(*kargs, **kwargs)
        Train a supervised model and return a model object.

        input must be a filepath. Should be UTF-8. 
        The input file must must contain at least one label per line. 

    train_unsupervised(*kargs, **kwargs)
        Train an unsupervised model and return a model object.
        input must be a filepath

In [ ]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

--2021-02-08 16:51:40--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2021-02-08 16:51:41--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6b985ef76ee9a5579f5a343049.dl.dropboxusercontent.com/cd/0/inline/BIjnzroZMXgxok2jRVojetAYSXFLERNtnBlcV2QUGXM6ooWdhbc9R2ne20ylIGcix5Iv477xn6KOHOEMfPiyoM6Uuw5QNHs4wUn6-fGLPXWq8w/file# [following]
--2021-02-08 16:51:41--  https://uc6b985ef76ee9a5579f5a343049.dl.dropboxusercontent.com/cd/0/inline/BIjnzroZMXgxok2jRVojetAYSXFLERNtnBlcV2QUGXM6ooWdhbc9R2ne20ylIGcix5Iv477xn6KOHOEMfPiyoM6Uuw5QNHs4wUn6-fGLPXWq8w/file
Resolving u

In [ ]:
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

--2021-02-08 16:51:46--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2021-02-08 16:51:47--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2f772f97de1b22787d390fc3ef.dl.dropboxusercontent.com/cd/0/inline/BIi1as2vqtpQvClFqPJr6lQiPwhL3Z2haB0-QPZQfMmJzX18DEiPMd0iOQVs8XVCEblb4L9mYPikHvC-jJwvGzAeiqdeAv4tZLujDJKaQ7pPEQ/file# [following]
--2021-02-08 16:51:47--  https://uc2f772f97de1b22787d390fc3ef.dl.dropboxusercontent.com/cd/0/inline/BIi1as2vqtpQvClFqPJr6lQiPwhL3Z2haB0-QPZQfMmJzX18DEiPMd0iOQVs8XVCEblb4L9mYPikHvC-jJwvGzAeiqdeAv4tZLujDJKaQ7pPEQ/file
Resolving u

In [ ]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [ ]:
len(df)

226834

Проведем стандартный препроцессинг:

In [ ]:
! pip install pymorphy2

In [ ]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

m = pymorphy2.MorphAnalyzer()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [ ]:
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "    

In [ ]:
def clean_text(text):
    return lemmatize(words_only(text))

In [ ]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


,text,label,lemmas
0,"@first_timee хоть я и школота, но поверь, у на...",positive,first_timee хоть я и школотый но поверь у мы т...
1,"Да, все-таки он немного похож на него. Но мой ...",positive,да всё таки он немного похожий на он но мой ма...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,rt katiacheh: ну ты идиотка) я испугаться за т...
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,rt digger : кто то в угол сидеть и погибать от...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,irina_dyshkant вот что значит страшилка :d но ...


Запишем полученные данные в формате для обучения классификатора:

In [ ]:
X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


In [ ]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    
with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [ ]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.8961606284065405
R@1: 0.8961606284065405
Number of examples: 74856
